<a href="https://colab.research.google.com/github/joezaarour/Repo1/blob/main/lab3_AV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Financial Data Analysis with Alpha Vantage
**Objective:** Fetch, analyze, and visualize financial data using Alpha Vantage API.

Documentation : https://pypi.org/project/alpha-vantage/
---

## 0. Introduction & Setup

### A. Why Alpha Vantage?
- Free tier available
- Covers stocks, forex, crypto, and more
- Well-documented and reliable

### B. Get Your API Key
1. Sign up at [Alpha Vantage](https://www.alphavantage.co/support/#api-key).
2. Copy your free API key.

### C. Install Libraries
```bash
!pip install alpha_vantage
```

### D. Import Libraries

In [ ]:
!pip install alpha_vantage

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.fundamentaldata import FundamentalData
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import linregress

---

## 1. Basic Data Fetching

---



### A. Fetch Daily Stock Data

In [ ]:
# Replace with your API key
api_key = 'YOUR_API_KEY'
ts = TimeSeries(key=api_key, output_format='pandas')
aapl_daily, meta_data = ts.get_daily('AAPL')
aapl_daily = aapl_daily.rename(columns={'1. open': 'Open', '2. high': 'High', '3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'})
aapl_daily = aapl_daily.sort_index(ascending=True)
aapl_daily.head()

# Exercise:
Write a function that takes a dataframe as parameter and automatically renames the columns

### B. Basic Analysis & Visualization
* Plot the closing prices
* Calculate and plot the moving averages for 20 and 50 dates windows
  * hint: look at the 'rolling' and 'mean' functions in pandas
* Calculate and plot daily returns

In [ ]:
# Plot closing price

In [ ]:
# Calculate Moving averages

In [ ]:
# Calculate daily returns


---

## 2. Advanced Data Types

### A. Fetch Forex Data
* Get daily currency exchanges daily from EUR to USD
  * Use your function to format the dataframe
  * Plot the result

In [ ]:
# Fetch data


In [ ]:
# Plot EUR/USD


### B. Fetch Crypto Data
* Retrieve Bitcoin digital currence ('BTC') in the 'USD' market
* Plot the closing prices

In [ ]:
# Fetch BTC data

In [ ]:
# Plot BTC/USD


### C. Fetch Technical Indicators (SMA, RSI, MACD)
* Retrieve the relative strength index (RSI) for Apple ('AAPL')
* Plot the result

In [ ]:
# Fetch RSI

In [ ]:
# Plot RSI


---

## 3. Data Analysis & Visualization

### A. Correlation Analysis
* Retrieve some more daily stock data for comparison
  * Compute daily return
  * merge the two 'Daily return' columns from your two dataframes using pd.merge (relevant parameters: left_index=True, right_index=True, suffixes=('_sufx1', '_sufx2')
  * Drop all NaN rows and display result
  * Compute correlation using corr() on the merged df
  * Visualize the correlation using seaborn

In [ ]:
# Fetch  data for comparison

# Merge daily returns


In [ ]:
# Calculate correlation


In [ ]:
# Plot scatter and regression line
